## Install from GIT repository:

In [ ]:
# !pip install git+https://github.com/wasimar/ChatTTS.git
# Install english language Normalizer
# !pip install nemo_text_processing

In [1]:
import torch
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

import ChatTTS
from IPython.display import Audio

## Load Models

In [2]:
chat = ChatTTS.Chat()
chat.load_models()

# Use force_redownload=True if the weights updated.
# chat.load_models(force_redownload=True)

# If you download the weights manually, set source='locals'.
# chat.load_models(source='local', local_path='YOUR LOCAL PATH')

## Inference

### Batch infer

In [3]:
texts = ["So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with.",]*3 \
        + ["我觉得像我们这些写程序的人，他，我觉得多多少少可能会对开源有一种情怀在吧我觉得开源是一个很好的形式。现在其实最先进的技术掌握在一些公司的手里的话，就他们并不会轻易的开放给所有的人用。"]*3     
        
wavs = chat.infer(texts)

INFO:ChatTTS.core:All initialized.
 47%|████▋     | 960/2048 [00:07<00:08, 133.25it/s]


In [4]:
Audio(wavs[0], rate=24_000, autoplay=True)

In [5]:
Audio(wavs[3], rate=24_000, autoplay=True)

### Custom params

In [6]:
params_infer_code = {'prompt':'[speed_5]', 'temperature':.3}
params_refine_text = {'prompt':'[oral_2][laugh_0][break_6]'}

wav = chat.infer('四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片等，每样都让人垂涎三尺。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

INFO:ChatTTS.core:All initialized.
 22%|██▏       | 452/2048 [00:03<00:11, 140.51it/s]


In [7]:
Audio(wav[0], rate=24_000, autoplay=True)

### fix random speaker

In [8]:
rand_spk = chat.sample_random_speaker()
params_infer_code = {'spk_emb' : rand_spk, }

wav = chat.infer('四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

In [9]:
Audio(wav[0], rate=24_000, autoplay=True)

### Two stage control

In [10]:
text = "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with."
chat.infer(text, refine_text_only=True)

INFO:ChatTTS.core:All initialized.
 23%|██▎       | 87/384 [00:00<00:01, 150.60it/s]


['so we found being competitive and collaborative [uv_break] was a huge way of staying [uv_break] motivated towards our goals, [uv_break] so [uv_break] one person to call [uv_break] when you fall off, [uv_break] one person who [uv_break] gets you back [uv_break] on then [uv_break] one person [uv_break] to actually do the activity with.']

In [11]:
text = 'so we found being competitive and collaborative [uv_break] was a huge way of staying [uv_break] motivated towards our goals, [uv_break] so [uv_break] one person to call [uv_break] when you fall off, [uv_break] one person who [uv_break] gets you back [uv_break] on then [uv_break] one person [uv_break] to actually do the activity with.'
wav = chat.infer(text, skip_refine_text=True)

INFO:ChatTTS.core:All initialized.
 49%|████▊     | 995/2048 [00:07<00:07, 141.85it/s]


## LLM Call

In [12]:
from ChatTTS.experimental.llm import llm_api

API_KEY = ''
client = llm_api(api_key=API_KEY,
        base_url="https://api.deepseek.com",
        model="deepseek-chat")

In [13]:
user_question = '四川有哪些好吃的美食呢?'
text = client.call(user_question, prompt_version = 'deepseek')
print(text)
text = client.call(text, prompt_version = 'deepseek_TN')
print(text)

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


四川美食可多了, 有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片、串串香、龙抄手、宜宾燃面、乐山钵钵鸡等, 每样都让人垂涎三尺。


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片、串串香、龙抄手、宜宾燃面、乐山钵钵鸡等，每样都让人垂涎三尺。


In [14]:
params_infer_code = {'spk_emb' : rand_spk, 'temperature':.3}

wav = chat.infer(text, params_infer_code=params_infer_code)

INFO:ChatTTS.core:All initialized.
 32%|███▏      | 647/2048 [00:04<00:09, 140.27it/s]
